<a href="https://colab.research.google.com/github/Medynal/Pollution/blob/main/App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip -q install streamlit
print('Done!')

Done!


In [11]:
import streamlit as st
import pandas as pd
import numpy as np
import joblib
from datetime import datetime
import gdown
from PIL import Image
import plotly.express as px
import re




In [14]:
url= ['https://drive.google.com/file/d/1MU76NuuLhcYoFjwkKE4ZiBK-gJuTWsL_/view?usp=sharing',
      'https://drive.google.com/file/d/1WYtQPmrYsOiwCuJ6yi3Z4Rh8qQouspRG/view?usp=sharing',
      'https://drive.google.com/file/d/1PMArX4fNSW1mY15bugIc7qXRl63nV-7e/view?usp=sharing',
      'https://drive.google.com/file/d/1LVCA5L2pNAIsYe1O63ODqAtjAtWzKBUa/view?usp=sharing']

pattern = r'd/([^/]+)/view'
for link in url:
    match = re.search(pattern, link)
    if match:
      downloadable_link= f'https://drive.google.com/uc?id={match.group(1)}'
      gdown.download(downloadable_link)
print ('download complete!' )


Downloading...
From: https://drive.google.com/uc?id=1MU76NuuLhcYoFjwkKE4ZiBK-gJuTWsL_
To: /content/aqi_regressor (1).pkl
100%|██████████| 10.1M/10.1M [00:00<00:00, 46.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1WYtQPmrYsOiwCuJ6yi3Z4Rh8qQouspRG
To: /content/aqib_classifier.pkl
100%|██████████| 56.6M/56.6M [00:00<00:00, 112MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PMArX4fNSW1mY15bugIc7qXRl63nV-7e
To: /content/bucket_encoder (1).pkl
100%|██████████| 534/534 [00:00<00:00, 646kB/s]
Downloading...
From: https://drive.google.com/uc?id=1LVCA5L2pNAIsYe1O63ODqAtjAtWzKBUa
To: /content/images.jpeg
100%|██████████| 7.65k/7.65k [00:00<00:00, 19.8MB/s]

download complete!


In [15]:
#loading Models
aqi_model= joblib.load('/content/aqi_regressor (1).pkl' )
bucket_model = joblib.load('/content/aqib_classifier.pkl')
bucket_encoder = joblib.load('/content/bucket_encoder (1).pkl')



In [16]:
image = Image.open('/content/images.jpeg')
st.image(image, width= 400)


2025-12-06 17:55:32.449 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-06 17:55:32.456 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-06 17:55:32.457 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-06 17:55:32.459 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [17]:
# page configuration
st.set_page_config(page_title='AQI Bucket Predictor',layout='wide')
st.title('AQI Bucket Prediction App')
col1 = st.sidebar
col2, col3 = st.columns((2,1))

2025-12-06 17:55:40.746 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-06 17:55:40.749 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-06 17:55:40.751 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-06 17:55:40.754 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-06 17:55:40.755 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-06 17:55:40.756 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-06 17:55:40.757 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [18]:
#Sidebar Section
col1.header('About this App')
col1.write("""
This app predicts:
- **AQI Bucket (Good, Moderate, Poor...)**
based on pollutant levels and a selected city and date.
""")

2025-12-06 17:55:49.927 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-06 17:55:49.930 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-06 17:55:49.931 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-06 17:55:49.933 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-06 17:55:49.934 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-06 17:55:49.935 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
col2.subheader('Input Parameters')

city = col2.selectbox('Select City', ['Ahmedabad', 'Aizawl', 'Amaravati', 'Amritsar', 'Bengaluru',
       'Bhopal', 'Brajrajnagar', 'Chandigarh', 'Chennai', 'Coimbatore',
       'Delhi', 'Ernakulam', 'Gurugram', 'Guwahati', 'Hyderabad',
       'Jaipur', 'Jorapokhar', 'Kochi', 'Kolkata', 'Lucknow', 'Mumbai',
       'Patna', 'Shillong', 'Talcher', 'Thiruvananthapuram',
       'Visakhapatnam'])

date = col2.date_input('Select Date')
year = date.year
month = date.month
day = date.day

# Cyclic encodings
month_sin = np.sin(2 * np.pi * month / 12)
month_cos = np.cos(2 * np.pi * month / 12)
day_sin = np.sin(2 * np.pi * day / 31)
day_cos = np.cos(2 * np.pi * day / 31)

# POLLUTANTS
pollutants = ['PM2.5','PM10','NO','NO2','NOx','NH3','CO','SO2','O3','Benzene','Toluene','Xylene']

pollutant_values = {}
for p in pollutants:
    pollutant_values[p] = col2.number_input(f"Enter {p} value", min_value=0.0)

input_data = pd.DataFrame([{
    "City": city,
    "year": year,
    "month_sin": month_sin,
    "month_cos": month_cos,
    "day_sin": day_sin,
    "day_cos": day_cos,
    **pollutant_values}])

if col2.button("Predict"):

    # Predict AQI
    predicted_aqi = aqi_model.predict(input_data)[0]

    # Add predicted AQI into features for bucket model
    input_data["AQI"] = predicted_aqi

    # Predict AQI Bucket
    predicted_bucket = bucket_model.predict(input_data)[0]
    predicted_bucket_label = bucket_encoder.inverse_transform([predicted_bucket])[0]

    # Show results
    col2.success(f"Predicted AQI: {predicted_aqi:.2f}")
    col2.info(f"Predicted AQI Bucket: **{predicted_bucket_label}**")




In [ ]:
col3.subheader("Pollutant Levels Chart")

pollutant_df = pd.DataFrame({'Pollutant': list(pollutant_values.keys()),
    "Value": list(pollutant_values.values())})

fig = px.bar(
    pollutant_df,
    x="Pollutant",
    y="Value",
    title="Input Pollutant Levels",
    text="Value")

fig.update_layout(
    xaxis_tickangle=-90,
    height=450)

col3.plotly_chart(fig, use_container_width=True)